# Visualização LDA

Utilizaremos a distribuição de topicos de documentos como o vetor embedding do documento.

## 1. Preprocessamento do Texto
Usamos o dataset de sinopsis de filme como nossos documentos e removemos palavras raras e palavras comuns com base na sua frequência no documento. Removemos palavras que aparecem em menos de 2 documentos ou em mais de 30% dos documentos.

In [1]:
import pandas as pd
import re
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary

# read data
dataframe = pd.read_csv('movie_plots.csv')

# remove stopwords and punctuations
def preprocess(row):
    return strip_punctuation(remove_stopwords(row.lower()))
    
dataframe['Plots'] = dataframe['Plots'].apply(preprocess)

# Convert data to required input format by LDA
texts = []
for line in dataframe.Plots:
    lowered = line.lower()
    words = re.findall(r'\w+', lowered, flags = re.UNICODE)# | re.LOCALE
    texts.append(words)
    
# Create a dictionary representation of the documents.
dictionary = Dictionary(texts)

# Filter out words that occur less than 2 documents, or more than 30% of the documents.
dictionary.filter_extremes(no_below=2, no_above=0.3)

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(text) for text in texts]

## 2. Treinamos o modelo LDA

In [2]:
# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 50
iterations = 200
eval_every = None

# Train model
model = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, chunksize=chunksize, alpha='auto', 
                          eta='auto', iterations=iterations, num_topics=num_topics, passes=passes, 
                          eval_every=eval_every)

## 3. Distribuição de topicos de documentos
Usamos o método get_document_topics que realiza a infrencia de a distribuição de topicos de um documento. Retorna uma lista de tupa (topic_id, probability)

In [3]:
# Get document topics
all_topics = model.get_document_topics(corpus, minimum_probability=0)
all_topics[2]

[(0, 0.00042185804),
 (1, 0.00047327485),
 (2, 0.0003953588),
 (3, 0.00044829352),
 (4, 0.00044607325),
 (5, 0.00038524263),
 (6, 0.99617916),
 (7, 0.00040969712),
 (8, 0.00048974983),
 (9, 0.00035133513)]

## 4. Preparando os arquivos de entrada para TensorBoard

In [4]:
from tensorboard_helper import CreateTensorboardData

In [5]:
vectors = [ [topics[1] for topics in doc_topics] for doc_topics in all_topics]
metadatos = [dataframe.Titles.values , dataframe.Genres.values]

In [6]:
CreateTensorboardData(tensor_filename="doc_lda", 
                      vectors=vectors, 
                      metadatos=metadatos,
                      colnames=["Titles","Genres"])

Arquivo com o Tensor 2D foi salvado em: doc_lda_tensor.tsv
Arquivo com o Tensor de metadatos foi salvado em: doc_lda_metadata.tsv


Baixar arquivos de vetores : <a href="doc_lda_tensor.tsv" download="w3logo"> doc_lda_tensor.tsv </a>

Baixar arquivos com os metadatos: <a href="doc_lda_metadata.tsv" download="w3logo"> doc_lda_metadata.tsv </a>

**Visualizando usando PCA ...**

Levar os arquivos para http://projector.tensorflow.org/.

Como podemos ver, há muitos pontos que se agrupam nos cantos. Os documentos nos cantos pertencem principalmente a um único tópico (portanto, o peso grande em uma única dimensão e outras dimensões têm peso aproximadamente zero). Você pode modificar o arquivo de metadados conforme explicado abaixo para ver os pesos das dimensões juntamente com o título do filme.

Agora, vamos anexar os tópicos com maior probabilidade (topic_id, topic_probability) ao título do documento, para explorar quais tópicos os domínios ou bordas do cluster pertencem de forma dominante. Para isso, precisamos sobrescrever o arquivo de metadados conforme abaixo:

In [7]:
tensors = []
for doc_topics in all_topics:
    doc_tensor = []
    for topic in doc_topics:
        if round(topic[1], 3) > 0:
            doc_tensor.append((topic[0], float(round(topic[1], 3))))
            
    # sort topics according to highest probabilities
    doc_tensor = sorted(doc_tensor, key=lambda x: x[1], reverse=True)
    # store vectors to add in metadata file
    tensors.append(doc_tensor[:5])

In [10]:
tensors_str = [ str(" ".join([title, str(x)])) for title, x in zip(dataframe.Titles.values, tensors)]
metadatos = [tensors_str, dataframe.Genres.values]

CreateTensorboardData(tensor_filename="doc_lda", 
                      vectors=vectors, 
                      metadatos=metadatos,
                      colnames=["Titles","Genres"])

Arquivo com o Tensor 2D foi salvado em: doc_lda_tensor.tsv
Arquivo com o Tensor de metadatos foi salvado em: doc_lda_metadata.tsv


Baixar arquivos de vetores : <a href="doc_lda_tensor.tsv" download="w3logo"> doc_lda_tensor.tsv </a>

Baixar arquivos com os metadatos: <a href="doc_lda_metadata.tsv" download="w3logo"> doc_lda_metadata.tsv </a>

**Visualizando usando T-SNE ...**

Levar os arquivos para http://projector.tensorflow.org/.

In [2]:
pwd

'/home/Tesla/ControlCursos/DeepLearning/WordEmbedding'

In [ ]:
model.show_topic(topicid=0, topn=15)

In [ ]:
import pyLDAvis.gensim

viz = pyLDAvis.gensim.prepare(model, corpus, dictionary)
pyLDAvis.display(viz)